In [1]:
from paddlenlp.transformers import BertModel, BertTokenizer,BertForMaskedLM
import paddle
import numpy as np
import ot

/home/ricardo/miniconda3/envs/PaddleEE/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = BertModel.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

[2022-05-25 12:03:27,060] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/bert-base-chinese/bert-base-chinese.pdparams
[2022-05-25 12:03:38,692] [    INFO] - Weights from pretrained model not used in BertModel: ['cls.predictions.decoder_weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.weight', 'cls.predictions.transform.bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.layer_norm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
[2022-05-25 12:03:39,322] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt


In [8]:
test_str1 = '恒大地产2012年拟裁员30%：1月4日，恒大地产（3333.HK）的一位副总裁向记者确认，公司内部发文，2012年将裁员30%。知情人士透露，按步骤初期只裁10%，因为2009年裁员的时候，矛盾冲突比较激烈。该副总称裁员的原因主要是项目销售问题。'
model_inputs1 = tokenizer(test_str1)
print(tokenizer.all_special_ids)
print(model_inputs1)
# model_outputs1 = model(**{k:paddle.to_tensor([v]) for (k,v) in model_inputs1.items()})

[100, 102, 0, 101, 103]
{'input_ids': [101, 101, 2608, 1920, 1765, 772, 8151, 2399, 2877, 6161, 1447, 8114, 110, 8038, 122, 3299, 125, 3189, 8024, 2608, 1920, 1765, 772, 8020, 10745, 8152, 119, 100, 8021, 4638, 671, 855, 1199, 2600, 6161, 1403, 6381, 5442, 4802, 6371, 8024, 1062, 1385, 1079, 6956, 1355, 3152, 8024, 8151, 2399, 2199, 6161, 1447, 8114, 110, 511, 4761, 2658, 782, 1894, 6851, 7463, 8024, 2902, 3635, 7752, 1159, 3309, 1372, 6161, 8108, 110, 8024, 1728, 711, 8170, 2399, 6161, 1447, 4638, 3198, 952, 8024, 4757, 4688, 1103, 4960, 3683, 6772, 4080, 4164, 511, 6421, 1199, 2600, 4917, 6161, 1447, 4638, 1333, 1728, 712, 6206, 3221, 7555, 4680, 7218, 1545, 7309, 7579, 511, 102, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

['恒', '大', '地', '产', '（', '333', '##3', '.', '[UNK]', '）']
['项', '目']
['销', '售', '问', '题']
['地', '产']


In [ ]:
pos_list1 = [
    [18,28],
    [103,105],
    [105,109],
    [20,22],
]
for pos in pos_list1:
    print(tokenizer.convert_ids_to_tokens(model_inputs1['input_ids'][pos[0]:pos[1]]))

entity_embs1 = []
for pos in pos_list1:
    temp_maxtrix = paddle.stack([model_outputs1[0][0,idx] for idx in range(pos[0],pos[1])])
    entity_embs1.append(paddle.max(temp_maxtrix,axis=0).numpy()) # 

In [5]:
test_str2 = '从2021年9月份开始，恒大全国各地项目接连停工率，交楼量不足1万套导致恒大的销量大幅下降；每天都有商票逾期，债券经被停止融资融业务，裁员60%'
model_inputs2 = tokenizer(test_str2)
model_outputs2 = model(**{k:paddle.to_tensor([v]) for (k,v) in model_inputs2.items()})

In [6]:
pos_list2 = [
    [34,43],
    [10,12],
    [18,22],
    [10,12],
]
for pos in pos_list2:
    print(tokenizer.convert_ids_to_tokens(model_inputs2['input_ids'][pos[0]:pos[1]]))

entity_embs2 = []
for pos in pos_list2:
    temp_maxtrix = paddle.stack([model_outputs2[0][0,idx] for idx in range(pos[0],pos[1])])
    entity_embs2.append(paddle.max(temp_maxtrix,axis=0).numpy()) 
    

['恒', '大', '的', '销', '量', '大', '幅', '下', '降']
['恒', '大']
['接', '连', '停', '工']
['恒', '大']


In [13]:
def _compute_wasserstein_distance(node_embeddings_matrice, sinkhorn=False, 
                                   sinkhorn_lambda=0.01,isImport=False):
    '''
    Generate the Wasserstein distance matrix for the docs embedded 
    in label_sequences
    '''
    # n is the num of docs 
    n = len(node_embeddings_matrice) 
    
    
    M = np.zeros((n,n)) # 

    for i in range(len(node_embeddings_matrice)):
        node_embeddings_matrice[i] = np.nan_to_num(node_embeddings_matrice[i],posinf=1.0,neginf=1e-7,nan=0)
    # Iterate over pairs of docs
    for doc_index_1, doc_1 in enumerate(node_embeddings_matrice):
        for doc_index_2, doc_2 in enumerate(node_embeddings_matrice[doc_index_1:]):
            
            # Get cost matrix
            costs = ot.dist(doc_1, doc_2, metric='sqeuclidean') # (n,m) (6,4)
            # doc_1 has n metric -》 6, bert emb
            # doc_2 has m metric -> 4, bert emb
            
            doc1_dis = np.ones(doc_1.shape[0])/doc_1.shape[0] # uniform dis 1/6 
            doc2_dis = np.ones(doc_2.shape[0])/doc_2.shape[0] # uniform ids 1/4

            if sinkhorn:
                mat = ot.sinkhorn(doc1_dis, doc2_dis, costs, sinkhorn_lambda, numItermax=50) # sinkhorn approx
                M[doc_index_1, doc_index_2 + doc_index_1] = np.sum(np.multiply(mat, costs))
            else:
                M[doc_index_1, doc_index_2 + doc_index_1] = ot.emd2(doc1_dis,doc2_dis, costs)
                    
    M = (M + M.T)
    return M # 是一个n*n的matric，记录了任意两个 doc 之间的相似度

In [11]:
entity_embs1 = np.array(entity_embs1)
entity_embs2 = np.array(entity_embs2)

In [14]:
print(_compute_wasserstein_distance([entity_embs1,entity_embs2]))

[[  0.         376.97387695]
 [376.97387695   0.        ]]





有个问题是如何从 distance -> similarity probability matrix -> similairity matrix                           
或者添加一个可以训练的阈值


// 这样才可以加loss

